<a href="https://colab.research.google.com/github/Zooshi/llms/blob/main/First_Transformer_in_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
from transformers import GPT2Tokenizer, GPTSw3Tokenizer
import torch.nn.functional as F

In [15]:
class Model(nn.Module):
  def __init__(self,vocab_size,d_model,nhead,num_layers):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size,d_model)
    self.transformer = nn.Transformer(d_model = d_model, nhead=nhead,num_encoder_layers=num_layers)
    self.fc = nn.Linear(d_model,vocab_size)

  def forward(self,x):
    x = self.embedding(x)
    out = self.transformer(x,x)
    out = self.fc(out)
    return out

In [3]:
vocab_size = 10000  # Example vocabulary size
d_model = 256      # Model's hidden dimension
nhead = 8          # Number of attention heads
num_layers = 6     # Number of transformer layers


In [16]:
model = Model(vocab_size,d_model,nhead,num_layers)
model.eval()

Model(
  (embedding): Embedding(10000, 256)
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-5): 6 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
          )
          (linear1): Linear(in_features=256, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=2048, out_features=256, bias=True)
          (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
      )
      (norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    )
    (decoder): TransformerDecoder(
      (layers): ModuleList(
        (0-5): 6 x TransformerDecoderLayer(
          (s

In [5]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [35]:
def generate_text(prompt,max_length=50, temperature = 0.1):
  with torch.no_grad():
    tokenized = torch.tensor([tokenizer.encode(prompt)])
    #print(tokenized)
    for _ in range(max_length):
      logits = model(tokenized)
      #print(logits)
      logits = logits[:,-1,:]
      next_token  =torch.multinomial(F.softmax(logits,dim=-1),num_samples=1)
      tokenized = torch.cat((tokenized,next_token),dim=1)
      print
    for i in tokenized:
      if i>50000:
        i = 50000
    generated_text = tokenizer.decode(tokenized[0].tolist(),skip_special_tokens= True)
    return generated_text

In [36]:
promptinput = "Whey protein is"

In [37]:
generated_output = generate_text(promptinput)

RuntimeError: ignored

In [41]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import GPT2Tokenizer

class GPT2Simple(nn.Module):
    def __init__(self, vocab_size, d_model, nhead, num_layers):
        super(GPT2Simple, self).__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.transformer = nn.Transformer(
            d_model=d_model, nhead=nhead, num_encoder_layers=num_layers
        )
        self.fc = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        output = self.transformer(x, x)
        output = self.fc(output)
        return output

# Parameters
vocab_size = 100000  # Example vocabulary size
d_model = 256      # Model's hidden dimension
nhead = 8          # Number of attention heads
num_layers = 6     # Number of transformer layers

# Create the model
model = GPT2Simple(vocab_size, d_model, nhead, num_layers)

# Load the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Set the model in evaluation mode
model.eval()

# Check if GPU is available
device = "cpu" #torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define a function to generate text based on a prompt
def generate_text(prompt, max_length=50, temperature=1.0):
    with torch.no_grad():
        tokenized_prompt = torch.tensor([tokenizer.encode(prompt)])
        tokenized_prompt = tokenized_prompt.to(device)
        output = tokenized_prompt

        for _ in range(max_length):
            logits = model(output)  # Get logits for the next token
            logits = logits[:, -1, :] / temperature  # Apply temperature
            next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
            if next_token == None:
              next_token = 999
            output = torch.cat((output, next_token), dim=1)
            #print(output[0].tolist())
            output = torch.clip(output,0,50000)
        generated_text = tokenizer.decode(output[0].tolist(), skip_special_tokens=True)
        return generated_text

# Provide a prototype or prompt
prototype = "Whey Protein is"

# Generate text using the prototype
generated_output = generate_text(prototype, max_length=10, temperature=0.1)

# Print the generated output
print("Generated Output:", generated_output)

Generated Output: Whey Protein is Bridges gridsomersUnix ApocalypseResp Toronto gridsomers welcomed


In [34]:
tokenizer.decode([10842, 88, 31702, 318, 43305, 10889, 50023, 29695, 19378, 29695])

'Whey Protein is Shiv Sure cringe parody positioned parody'

In [31]:
tokenizer.encode("How are you")

[2437, 389, 345]

In [33]:
tokenizer.vocab_size

50257

In [38]:
ex = torch.tensor([10842, 88, 31702, 318, 43305, 10889, 50023, 77561, 69698, 29695, 19378, 90220, 88925, 29695])
ex

tensor([10842,    88, 31702,   318, 43305, 10889, 50023, 77561, 69698, 29695,
        19378, 90220, 88925, 29695])

In [40]:
torch.clip(ex,0,50000)

tensor([10842,    88, 31702,   318, 43305, 10889, 50000, 50000, 50000, 29695,
        19378, 50000, 50000, 29695])